In [1]:
!pip install transformers
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import torch
import os

In [3]:
# Set the API key
os.environ['ACCESS_KEY'] = ''

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
api_key = os.getenv('ACCESS_KEY')
tokenizer = AutoTokenizer.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)
model = AutoModelForCausalLM.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

df = pd.read_csv('/content/cti-mcq.tsv', sep='\t')
df.head()

In [ ]:
import csv

def generate_response(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def qcm_chat(df, model, tokenizer):
    results = []

    for index, row in df.iterrows():
        question = row['Question']
        options = {
            'A': row['Option A'],
            'B': row['Option B'],
            'C': row['Option C'],
            'D': row['Option D']
        }
        correct_answer = row['GT']
        prompt = f"Question: {question}\nOptions:\nA) {options['A']}\nB) {options['B']}\nC) {options['C']}\nD) {options['D']}\nPlease choose the correct option:"

        response = generate_response(prompt, model, tokenizer)
        model_answer = response.strip().upper()[0]  # Simple extraction logic

        is_correct = model_answer == correct_answer
        results.append({
            'Question': question,
            'Option A': options['A'],
            'Option B': options['B'],
            'Option C': options['C'],
            'Option D': options['D'],
            'Model Answer': model_answer,
            'Correct Answer': correct_answer,
            'Is Correct': is_correct,
            'Model Response': response
        })

        print(f"Processed question {index + 1}/{len(df)}")

    # Save results to CSV file
    keys = results[0].keys()
    with open('model_responses.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)

    print("Results saved to model_responses.csv")

if __name__ == "__main__":
    qcm_chat(df, model, tokenizer)
